In [1]:
import pyprind
import pandas as pd
import os
import re
import numpy as np
import scipy.io as io
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split
import tensorflow as tf

E:\Data\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def rmse(predictions, targets):
    def_mean = predictions.shape[0]
    sum_errs = np.sum((predictions - targets) ** 2)
    return np.sqrt(sum_errs/def_mean)

In [3]:
mat = io.loadmat("Col_Spec")
spec = mat['Col_Spec']
colors = io.loadmat("colors")
rgba = colors['RGB_A']
rgbf = colors['RGB_F']
rgbd = colors['RGB_D65']
colmat = np.concatenate((rgbd, rgbf, rgba), axis=1)
specmat = np.concatenate((spec, spec, spec), axis=1)
x = np.arange(1,102)

In [4]:
spectrum_num = 100

In [5]:
X_train, X_test, y_train, y_test = train_test_split(np.transpose(colmat), np.transpose(specmat), test_size=0.333)

In [6]:
np.random.seed(123)
def create_batch_generator(x, y=None, num_vals=3, 
                           num_step=101, batch_size=64):
    n_batches = len(x)//batch_size
    x = x[:n_batches*batch_size, :num_vals]
    if y is not None:
        y = y[:n_batches*batch_size, :num_step]
    for ii in range(0, len(x), batch_size):
        if y is not None:
            xout = [None]
            xout.extend(list(x[ii:ii+batch_size, :num_vals]))
            yield np.array(xout), y[ii:ii+batch_size, :num_step]
        else:
            xout.extend(list(x[ii:ii+batch_size, :num_vals]))
            yield np.array(xout)

In [7]:
class HSE_RNN(object):
    def __init__(self, lstm_size=256,
                 num_layers=1, batch_size=64,
                 num_steps = 101,
                 value_size=3,
                 learning_rate=0.0001):
        self.lstm_size = lstm_size
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.num_steps = num_steps
        self.value_size = value_size
    
        self.g = tf.Graph()
        with self.g.as_default():
            tf.set_random_seed(123)
            self.build()
            self.saver = tf.train.Saver()
            self.init_op = tf.global_variables_initializer()
            
    
    def build(self):
        tf_x = tf.placeholder(tf.float64,
                             shape=(None, self.batch_size, self.value_size),
                             name='tf_x')
        tf_y = tf.placeholder(tf.float64,
                             shape=(self.batch_size, self.num_steps),
                             name='tf_y')
        tf_keepprob = tf.placeholder(tf.float64,
                                    name='tf_keepprob')
        cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(
                                            tf.contrib.rnn.BasicLSTMCell(self.lstm_size),
                                            output_keep_prob=tf_keepprob)
                                            for i in range(self.num_layers)])
        self.initial_state = cells.zero_state(self.batch_size, tf.float64)
        print('   << initial state >> ', self.initial_state)
        lstm_outputs, self.final_state = tf.nn.dynamic_rnn(cells, tf_x, initial_state=self.initial_state)
        print('\n <<  lstm output  >> ', lstm_outputs)
        print('\n << final state   >> ', self.final_state)
        logits=tf.layers.dense(inputs=lstm_outputs[:, -1],
                             units=self.num_steps, activation=None,
                             name='logits')
        logits=tf.squeeze(logits, name='logits_squeezed')
        print('\n << logits        >> ', logits)
        y_proba = tf.nn.sigmoid(logits, name='probabilities')
        predictions = {
            'probabilities': y_proba,
            'labels': tf.cast(tf.round(y_proba), tf.int32,
                              name='labels')
        }
        print('\n << predictions   >> ', predictions)
        #cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf_y,
        #                                                             logits=logits),
        #                     name='cost')
        cost = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(tf_y, logits))), name='cost')
        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        train_op = optimizer.minimize(cost, name='train_op')
        
   
    def train(self, X_train, y_train, num_epochs):
        with tf.Session(graph=self.g) as sess:
            sess.run(self.init_op)
            iteration = 1
            for epoch in range(num_epochs):
                state = sess.run(self.initial_state)
                for batch_x, batch_y in create_batch_generator(X_train, y_train,
                                                              self.value_size, 
                                                               self.num_steps,
                                                               self.batch_size):
                    print(batch_x.shape)
                    print(batch_x)
                    feed = {'tf_x:0': batch_x,
                           'tf_y:0': batch_y,
                           'tf_keepprob:0': 0.5,
                           self.initial_state: state}
                    loss, _, state = sess.run(
                    ['cost:0', 'train_op',
                    self.final_state],
                    feed_dict=feed)
                    if iteration % 20 == 0:
                        print("Epoch: %d/%d Iteration: %d "
                             "| Train loss: %.5f" % (
                             epoch + 1, num_epochs, 
                             iteration, loss))
                    iteration += 1
                if (epoch + 1) % 10 == 0:
                    slef.saver.save(sess,
                                   "model/sentiment-%d" % epoch)
    
    def predict(self, X_data, return_proba=False):
        preds=[]
        with tf.Session(graph=self.g) as sess:
            self.saver.restore(
            sess, tf.train.latest_checkpoint('./model/'))
            test_state = sess.run(self.initial_state)
            for ii, batch_x in enumerate(
            create_batch_generator(
            X_data, None, self.value_size,
                self.num_steps,
                batch_size=self.batch_size), 1):
                feed = {'tf_x:0': batch_x,
                       'tf_keepprob:0': 1.0,
                       self.initial_state: test_state}
                if return_proba:
                    pred, test_state = sess.run(
                    ['probabilities:0', self.final_state],
                    feed_dict=feed)
                else:
                    perd, test_state = sess.run(
                    ['labels:0', self.final_state],
                    feed_dict=feed)
                preds.append(pred)
        return np.concatenate(preds)

In [8]:
rnn = HSE_RNN(lstm_size=101,
                   num_layers=2,
                   batch_size=100,
                   num_steps=101,
                   learning_rate=0.001)

   << initial state >>  (LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros:0' shape=(100, 101) dtype=float64>, h=<tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros_1:0' shape=(100, 101) dtype=float64>), LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState_1/BasicLSTMCellZeroState/zeros:0' shape=(100, 101) dtype=float64>, h=<tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState_1/BasicLSTMCellZeroState/zeros_1:0' shape=(100, 101) dtype=float64>))

 <<  lstm output  >>  Tensor("rnn/transpose_1:0", shape=(100, 100, 101), dtype=float64)

 << final state   >>  (LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_3:0' shape=(100, 101) dtype=float64>, h=<tf.Tensor 'rnn/while/Exit_4:0' shape=(100, 101) dtype=float64>), LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_5:0' shape=(100, 101) dtype=float64>, h=<tf.Tensor 'rnn/while/Exit_6:0' shape=(100, 101) dtype=float64>))

 << logits        >> 

In [9]:
rnn.train(X_train, y_train, num_epochs=40)

(101,)
[None array([23.4760101 , 25.04848485, 28.68754209])
 array([99.72743056, 73.23390152, 65.95496633])
 array([55.06404063, 20.52582683,  5.6215781 ])
 array([ 37.46482003, 111.06021909,  65.18984351])
 array([100.49556143,  38.09215168,   1.06825397])
 array([113.09852778,  96.26352778,  75.25675   ])
 array([91.42475694, 83.61715278, 22.08989583])
 array([35.21825397, 85.05576259, 81.1347136 ])
 array([90.96205717, 40.40455582, 16.51140633])
 array([54.92504085,  9.80547386,  0.58202614])
 array([36.81867284, 38.31801698, 40.1996142 ])
 array([107.25086914, 101.93241552,  56.78490474])
 array([46.43344429, 17.25432354, 28.91339532])
 array([3.93941288e+01, 2.57332071e+01, 5.68181818e-04])
 array([91.33104067, 77.5562201 , 14.79907629])
 array([106.56071099,  68.70252385,  20.70917975])
 array([62.17708333, 58.91945302, 25.93364198])
 array([119.37067793,  50.31512238,  14.22091103])
 array([107.26070783,  74.10951189,  55.77864692])
 array([142.82775915, 112.94335355, 151.236244

ValueError: setting an array element with a sequence.